In [62]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 250)

### Hyperpartisan

In [71]:
# !cd ..;python parse_tb.py

In [64]:
# res_df[(res_df.from_pretrained == 'bert-base-cased') & (res_df.input_seq_len == 818)]

In [65]:
paths = ['../results/hyp_grid.csv',
         '../results/hyp.csv',
         '../results/results_sbc/hyp_grid.csv'
        ]
         
         

In [47]:
res_df = pd.concat([pd.read_csv(p) for p in paths])
res_df_debug = pd.read_csv('../results/debug_hyp.csv')
res_df['sum_loss'] = False
res_df_debug['sum_loss'] = True
res_df = pd.concat((res_df, res_df_debug))
res_df.input_seg_size = res_df.input_seg_size.fillna(512)

In [48]:
res_df = res_df.drop_duplicates()
res_df.num_mem_tokens = res_df.num_mem_tokens.fillna(0)
res_df = res_df[res_df.f1 > 0]

In [49]:
gb_cols = ['from_pretrained', 'input_seg_size', 'input_seq_len', 'num_mem_tokens', 'lr', 'lr_scheduler', 'sum_loss']
metric_cols = ['f1', 'best_valid_f1']

group_df = res_df.groupby(gb_cols).mean()[metric_cols]
group_df['f1_std'] = res_df.groupby(gb_cols).std()[['f1']]
group_df['n_runs'] = res_df.groupby(gb_cols).count()['f1']
group_df['runs'] = res_df.groupby(gb_cols).agg({'f1':tuple}).f1.apply(lambda x: [round(i,2) for i in x])
# group_df

In [50]:
# group_df.to_csv('../results/grouped/hyp.csv')

### contract-nli

In [66]:
paths = [
        '../results/cnli.csv',
        '../results/contract_nli.csv',
         '../results/debug_cnli.csv',
        ]
         

In [72]:
res_df = pd.concat([pd.read_csv(p) for p in paths])
res_df.sum_loss = res_df.sum_loss.fillna(False)

In [73]:
# res_df_d = pd.read_csv('../results/cnli_double_seg.csv')
# res_df_d['double_segment'] = True
# res_df = pd.concat((pd.read_csv('../results/cnli.csv'), res_df_d))

# res_df.double_segment = res_df.double_segment.fillna(False)
res_df.input_seg_size = res_df.input_seg_size.fillna(512)

In [77]:
res_df = res_df[res_df.best_valid_exact_match > 70]

In [78]:
gb_cols = ['from_pretrained', 'input_seg_size', 'input_seq_len', 'num_mem_tokens','batch_size', 'lr', 'lr_scheduler','sum_loss']
metric_cols = ['best_valid_exact_match']
group_df = res_df.groupby(gb_cols).mean()[metric_cols]
group_df['std'] = res_df.groupby(gb_cols).std()[['best_valid_exact_match']]
group_df['n_runs'] = res_df.groupby(gb_cols).count()['best_valid_exact_match']
group_df['runs'] = res_df.groupby(gb_cols).agg({'best_valid_exact_match':tuple}).best_valid_exact_match.apply(lambda x: [round(i,2) for i in x])
group_df

best_valid_exact_match  \
from_pretrained           input_seg_size input_seq_len num_mem_tokens batch_size lr      lr_scheduler         sum_loss                           
bert-base-cased           512.0          918           50             1          0.00001 linear               False                  72.596157   
                                                                                                              True                   72.350677   
                                         968           25             1          0.00001 constant_with_warmup False                  72.211538   
                                                                                                              True                   73.653847   
                                                                                         linear               False                  72.211536   
                                                                                                              True                   70.865387   
                                                                                 0.00005 constant_with_warmup False                  71.442307   
                                                                                         linear               False                  72.211540   
                                         998           10             1          0.00001 constant_with_warmup False                  72.211538   
                                                                                                              True                   72.596153   
                                                                                         linear               False                  72.115384   
                                                                                                              True                   70.961540   
                                                                                 0.00005 constant_with_warmup False                  71.490387   
                                                                                         linear               False                  72.692310   
                                                                      16         0.00001 constant_with_warmup True                   72.254333   
                                         1377          50             1          0.00001 linear               False                  72.163464   
                                                                                                              True                   72.447014   
                                         1452          25             1          0.00001 constant_with_warmup True                   72.980769   
                                                                                         linear               True                   72.788460   
                                         1497          10             1          0.00001 constant_with_warmup True                   72.596153   
                                                                                         linear               True                   72.019230   
                                                                                 0.00005 linear               True                   72.692307   
                                         1936          25             1          0.00001 constant_with_warmup True                   71.346153   
                                         1996          10             1          0.00001 constant_with_warmup True                   72.596153   
                                                                                         linear               True                   72.307693   
facebook/bart-base        512.0          1002          10             1          0.00001 constant_with_warmup False                  71.778851   
                                

In [56]:
# group_df.to_csv('../results/grouped/cnli_sum_loss.csv')

In [79]:
# gb_cols = gb_cols[:-1]

In [83]:
# gb_df[~gb_df.sum_loss].shape

In [84]:
gb_df = group_df.reset_index()
gb_df_copy = gb_df.copy()
gb_df_copy.sum_loss = ~gb_df_copy.sum_loss

In [85]:
merged = gb_df.merge(gb_df_copy[gb_cols + ['best_valid_exact_match']], on=gb_cols, how='inner', suffixes=('', '_sum_loss'))
merged = merged[~merged.sum_loss]
merged['sum_loss_added'] = merged.best_valid_exact_match_sum_loss - merged.best_valid_exact_match
# merged

In [87]:
merged[['from_pretrained','input_seq_len', 'num_mem_tokens',
       'lr', 'lr_scheduler', 'best_valid_exact_match', 'sum_loss_added']].sort_values('sum_loss_added', ascending=False)

,from_pretrained,input_seq_len,num_mem_tokens,lr,lr_scheduler,best_valid_exact_match,sum_loss_added
32,t5-base,1002,10,0.00050,constant_with_warmup,71.820801,1.863767
36,t5-base,1002,10,0.00100,constant_with_warmup,71.580002,1.857498
28,t5-base,972,25,0.00100,linear,70.664700,1.803270
24,t5-base,972,25,0.00050,linear,70.905548,1.637856
38,t5-base,1002,10,0.00100,linear,72.206203,1.591846
2,bert-base-cased,968,25,0.00001,constant_with_warmup,72.211538,1.442309
14,microsoft/deberta-v3-base,998,10,0.00001,constant_with_warmup,73.493591,1.410255
34,t5-base,1002,10,0.00050,linear,71.387299,1.334167
12,microsoft/deberta-v3-base,918,50,0.00001,linear,72.355770,0.861958
16,microsoft/deberta-v3-base,998,10,0.00001,linear,73.750000,0.543943


### quality

In [212]:
# !cd ..;python parse_tb.py

In [219]:
res_df = pd.read_csv('../results/quality.csv')
# res_df = pd.concat((pd.read_csv('results/results_sbc/quality.csv'),))
res_df.input_seg_size = res_df.input_seg_size.fillna(512)

In [220]:
res_df = res_df[res_df.best_valid_exact_match > 10]

In [224]:
gb_cols = ['from_pretrained', 'input_seg_size', 'input_seq_len', 'num_mem_tokens', 'lr', 'lr_scheduler']
metric_cols = ['best_valid_exact_match']
group_df = res_df.groupby(gb_cols).mean()[metric_cols]
group_df['std'] = res_df.groupby(gb_cols).std()[['best_valid_exact_match']]
group_df['n_runs'] = res_df.groupby(gb_cols).count()['best_valid_exact_match']
group_df['runs'] = res_df.groupby(gb_cols).agg({'best_valid_exact_match':tuple}).best_valid_exact_match.apply(lambda x: [round(i,2) for i in x])
group_df

best_valid_exact_match  \
from_pretrained input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                           
t5-base         512.0          998           10             0.00005 linear                     28.395667   
                                                            0.00010 linear                     30.021550   

                                                                                       std  \
from_pretrained input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler             
t5-base         512.0          998           10             0.00005 linear        0.508137   
                                                            0.00010 linear        0.167795   

                                                                                  n_runs  \
from_pretrained input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler           
t5-base         512.0          998           10             0.00005 linear             3   
                                                            0.00010 linear             4   

                                                                                                         runs  
from_pretrained input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                               
t5-base         512.0          998           10             0.00005 linear              [27.85, 28.48, 28.86]  
                                                            0.00010 linear        [29.82, 30.11, 30.2, 29.96]